# Loading Data from NHC Hurricane Archive 

Data Accessible Here [https://www.nhc.noaa.gov/data/hurdat/]
- See format details in `format.pdf`

There are two types of rows for this dataset
- Type 1: AL092021, IDA, 40,
- Type 2: 20210829, 1655, L, HU, 29.1N, 90.2W, 130, 931, 130, 110, 80, 110, 70, 60, 40, 60, 45, 35, 20, 30, 10
   - Note: Date is YYYMMDD and Time is UTC

In [4]:
import pandas as pd
import numpy as np
from io import StringIO

In [37]:
filename = 'hurdat2-1851-2023-051124.txt'
missingVal = -999
basin='AL'
columnHeaders = ['id', 'name', 'entries', 'date', 'time', 'record_identifier', 'status', 'lat', 'lon', 'vmax', 'pres', '34ne', '34se', '34sw', '34nw', '50ne', '50se', '50sw', '50nw', '64ne', '64se', '64sw', '64nw', 'rmax']

In [55]:
# Reformatting input file to csv and creating a new file named 'data.csv'
with open(filename, 'r') as f_in, open('data.csv', 'w') as f_out:
    tmp = ""
    for line in f_in:
        line = line.replace(' ', '')  # Remove spaces
        if line.startswith(basin):
            tmp = line.strip()  # Store the basin line
        else:
            line = tmp + line  # Prepend the last stored basin
            f_out.write(line)  # Write filtered lines immediately

In [60]:
data = pd.read_csv('data.csv', header=None, names=columnHeaders)
data = data.drop('entries', axis=1)  
data

,id,name,date,time,record_identifier,status,lat,lon,vmax,pres,...,34nw,50ne,50se,50sw,50nw,64ne,64se,64sw,64nw,rmax
0,AL011851,UNNAMED,18510625,0,NaN,HU,28.0N,94.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,NaN,HU,28.0N,95.4W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,NaN,HU,28.0N,96.0W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,NaN,HU,28.1N,96.5W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2N,96.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54744,AL212023,TWENTY-ONE,20231023,1800,NaN,TD,11.5N,83.2W,25,1007,...,0,0,0,0,0,0,0,0,0,60
54745,AL212023,TWENTY-ONE,20231024,0,NaN,TD,12.2N,83.4W,25,1007,...,0,0,0,0,0,0,0,0,0,60
54746,AL212023,TWENTY-ONE,20231024,130,L,TD,12.4N,83.5W,25,1007,...,0,0,0,0,0,0,0,0,0,60
54747,AL212023,TWENTY-ONE,20231024,600,NaN,TD,13.0N,83.8W,25,1007,...,0,0,0,0,0,0,0,0,0,60
